<a href="https://colab.research.google.com/github/Diegoarmando24/Practica-2/blob/main/Ejercicio_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy.linalg import norm

# ================================
# FUNCIONES DE RESOLUCIÓN BÁSICA
# ================================

def SustitucionDelante(L, b):
    """
    Realiza sustitución hacia adelante para resolver Ly = b
    donde L es una matriz triangular inferior.
    """
    n = L.shape[0]
    x = np.zeros_like(b)
    for i in range(n):
        x[i] = (b[i] - np.dot(L[i, :i], x[:i])) / L[i, i]
    return x

def SustitucionAtras(U, b):
    """
    Realiza sustitución hacia atrás para resolver Ux = y
    donde U es una matriz triangular superior.
    """
    n = U.shape[0]
    x = np.zeros_like(b)
    for i in reversed(range(n)):
        x[i] = (b[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    return x

def LU_Completa(A):
    """
    Realiza la factorización LU de una matriz A sin pivoteo.
    Devuelve matrices L y U tal que A = LU.
    """
    n = A.shape[0]
    L = np.eye(n)
    U = np.copy(A)
    for i in range(n):
        for j in range(i+1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j] -= L[j, i] * U[i]
    return L, U

def ResolverLU(A, b):
    """
    Resuelve Ax = b usando la factorización LU sin pivoteo.
    """
    L, U = LU_Completa(A)
    y = SustitucionDelante(L, b)
    return SustitucionAtras(U, y)

# ====================================
# FUNCIÓN PARA RUTINA TIPO BANDA
# ====================================

def ResolverSistemaBanda(A, b):
    """
    Resuelve Ax = b utilizando una rutina adaptada para matrices con banda.
    Solo se hacen operaciones sobre entradas cercanas a la diagonal.
    """
    n = len(b)
    L = np.eye(n)
    U = np.copy(A)

    for i in range(n):
        for j in range(i + 1, min(n, i + 3)):  # Solo opera sobre banda (2 arriba de diagonal)
            factor = U[j, i] / U[i, i]
            L[j, i] = factor
            U[j, i:] -= factor * U[i, i:]

    y = SustitucionDelante(L, b)
    return SustitucionAtras(U, y)

# ====================================
# CONSTRUCCIÓN DE MATRICES A Y R
# ====================================

def construir_A(n):
    """
    Construye la matriz A (tipo banda) de tamaño n x n
    según la estructura del problema.
    """
    A = np.zeros((n, n))
    for i in range(n):
        if i == 0:
            A[i, i] = 9
            A[i, i+1] = -4
            A[i, i+2] = 1
        elif i == 1:
            A[i, i-1] = -4
            A[i, i] = 6
            A[i, i+1] = -4
            A[i, i+2] = 1
        elif i == n-2:
            A[i, i-2] = 1
            A[i, i-1] = -4
            A[i, i] = 5
            A[i, i+1] = -2
        elif i == n-1:
            A[i, i-2] = 1
            A[i, i-1] = -2
            A[i, i] = 1
        else:
            A[i, i-2] = 1
            A[i, i-1] = -4
            A[i, i] = 6
            A[i, i+1] = -4
            A[i, i+2] = 1
    return A

def construir_R(n):
    """
    Construye matriz R tal que A = R · Rᵗ
    (estructura especial dada en el problema).
    """
    R = np.zeros((n, n))
    for i in range(n):
        R[i, i] = 1
        if i + 1 < n:
            R[i, i+1] = -2
        if i + 2 < n:
            R[i, i+2] = 1
    R[0, 0] = 2  # Valor modificado según el problema
    return R

# ====================================
# FUNCIÓN PARA RESOLVER SISTEMA USANDO R · Rᵗ
# ====================================

def Resolver_R_RT(R, b):
    """
    Resuelve el sistema A·x = b usando la factorización R·Rᵗ.
    """
    A = R @ R.T
    Rt = R.T
    y = SustitucionDelante(Rt, b)
    x = SustitucionAtras(R, y)
    return x, A

# ====================================
# FUNCIÓN PRINCIPAL DE RESOLUCIÓN
# ====================================

def resolver_para(n):
    """
    Ejecuta los tres métodos de resolución para un tamaño de matriz n:
    (a) LU sin pivoteo, (b) rutina tipo banda y (c) R·Rᵗ.
    Muestra los primeros 10 valores de x y el residuo ||Ax - b||.
    """
    b = np.ones(n)
    A = construir_A(n)
    R = construir_R(n)

    # Método (a): LU sin pivoteo
    x_LU = ResolverLU(A, b)
    residuo_LU = norm(A @ x_LU - b)

    # Método (b): Adaptado a banda
    x_banda = ResolverSistemaBanda(A, b)
    residuo_banda = norm(A @ x_banda - b)

    # Método (c): R·Rᵗ
    x_rrt, A_check = Resolver_R_RT(R, b)
    residuo_rrt = norm(A_check @ x_rrt - b)

    # Mostrar resultados
    print(f"\n------ Resultados para n = {n} ------")
    print("\n(a) LU sin pivoteo:")
    print("Primeros 10 valores de x:", x_LU[:10])
    print("Residuo ||Ax - b||:", residuo_LU)

    print("\n(b) Rutina tipo banda:")
    print("Primeros 10 valores de x:", x_banda[:10])
    print("Residuo ||Ax - b||:", residuo_banda)

    print("\n(c) Factorización R·Rᵗ:")
    print("Primeros 10 valores de x:", x_rrt[:10])
    print("Residuo ||RRᵗx - b||:", residuo_rrt)

# ====================================
# EJECUCIÓN PARA DOS TAMAÑOS DE MATRIZ
# ====================================

resolver_para(100)   # Matriz A de tamaño 100 x 100
resolver_para(1000)  # Matriz A de tamaño 1000 x 1000
